# Parallel Implementation of SPHINCS+With GPUs

> the pdf file is available [here](../related_work/Parallel_Implementation_of_SPHINCS_With_GPUs.pdf)


## Abstract

Therefore, we present highly parallel and optimized implementations of SPHINCS+ _on various NVIDIA GPU architectures_ (Pascal, Turing, and Ampere). We discovered parts within the _internal processes_ of SPHINCS+ that could be parallelized and optimized them (e.g., leaf node generation and node merging process in MSS, subtree constructions in FORS, signature generation in WOTS+ and hypertree layer construction), leveraging the characteristics of _GPU architecture_ (e.g., warp-based execution and efficient memory access).
